**Authors**:

[`Giovanni Squillero`](https://github.com/squillero/computational-intelligence) `<giovanni.squillero@polito.it>`

[`Marcello Vitaggio`](https://github.com/Kalller/computational-intelligence) `<s318904@studenti.polito.it>`  

Free for personal or classroom use; see [`LICENSE.md`](https://github.com/Kalller/computational-intelligence/blob/main/LICENSE.md) for details.  

In the following code, the solution of Lab 1 is proposed from the code developed in class by Professor Squillero. The skeleton of the code is nearly identical except for the addition of the new heuristic function required by the lab.

In [87]:
from random import random
from math import ceil
from functools import reduce
from collections import namedtuple, deque
from queue import PriorityQueue

import numpy as np

In [88]:
State = namedtuple('State', ['taken', 'not_taken'])

def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i in state.taken],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(covered(state))

In [89]:
PROBLEM_SIZE = 20
NUM_SETS = 40
SETS = tuple(np.array([random() < 0.2 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"


##### Heuristic Function proposed solution 

The initial step of the function involves checking if all the elements have already been covered. In the event that they have, the function returns a value of 0, indicating that no further action is needed.

For each set that has not yet been included, the function proceeds to calculate the efficiency of each set in terms of covering the remaining elements. This efficiency is determined by comparing the number of elements in the set that remain uncovered to the total number of elements in the set. In cases where the set has no elements, the efficiency is set to 0 in order to prevent any potential division errors.

Once the efficiencies have been calculated, the function arranges the sets in descending order based on their efficiencies. This ordering ensures that the function prioritizes the selection of sets that have the ability to cover the greatest number of remaining elements.

Next, the function begins selecting sets from the sorted list until all elements have been covered. Throughout this process, the function keeps track of the total number of sets that have been selected.

Ultimately, the function provides an estimation of the number of sets that are required to achieve the desired goal state from the current state.

In [90]:
def h(state):
    already_covered = covered(state)
    if np.all(already_covered):
        return 0
    
    remaining_elements = np.logical_not(already_covered)
    uncovered_sets = [i for i in range(NUM_SETS) if i not in state.taken]
    
    efficiencies = []
    for i in uncovered_sets:
        set_sum = sum(SETS[i])
        if set_sum != 0:
            efficiency = sum(np.logical_and(SETS[i], remaining_elements))/set_sum
            
        else:
            efficiency = 0  
        efficiencies.append(efficiency)
    
    sorted_sets = [x for _, x in sorted(zip(efficiencies, uncovered_sets), reverse=True)]
    
    taken = 0
    elements_covered = np.copy(already_covered)
    while not np.all(elements_covered):
        elements_covered = np.logical_or(elements_covered, SETS[sorted_sets[taken]])
        taken += 1
    
    return taken

def f(state):
    return len(state.taken) + h(state)

In [91]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((f(state), state))

counter = 0
_, current_state = frontier.get()

while not goal_check(current_state):
    counter += 1
    for action in current_state[1]:
        new_state = State(
            current_state.taken ^ {action},
            current_state.not_taken ^ {action},
        )
        frontier.put((f(new_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter:,} steps ({len(current_state.taken)} tiles)")

Solved in 7 steps (5 tiles)
